In [3]:
import pandas as pd
data = pd.read_csv('CloudWatch_Traffic_Web_Attack.csv')

print(data.info())
print(data.head())

print(data.isnull().sum())
data = data.dropna()

if 'timestamp' in data.columns:
    data['timestamp'] = pd.to_datetime(data['timestamp'])

print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282 entries, 0 to 281
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   bytes_in             282 non-null    int64 
 1   bytes_out            282 non-null    int64 
 2   creation_time        282 non-null    object
 3   end_time             282 non-null    object
 4   src_ip               282 non-null    object
 5   src_ip_country_code  282 non-null    object
 6   protocol             282 non-null    object
 7   response.code        282 non-null    int64 
 8   dst_port             282 non-null    int64 
 9   dst_ip               282 non-null    object
 10  rule_names           282 non-null    object
 11  observation_name     282 non-null    object
 12  source.meta          282 non-null    object
 13  source.name          282 non-null    object
 14  time                 282 non-null    object
 15  detection_types      282 non-null    object
dtypes: int64

In [5]:
if 'packet_size' in data.columns:
    features = data[['packet_size']]
else:
    features = pd.DataFrame()
if 'timestamp' in data.columns:
    data['time_diff'] = data['timestamp'].diff().dt.total_seconds().fillna(0)
    features['time_diff'] = data['time_diff']

print(features.head())

Empty DataFrame
Columns: []
Index: []


In [11]:
from sklearn.ensemble import IsolationForest

if not features.empty:
    isolation_forest = IsolationForest(contamination=0.01) 
    isolation_forest.fit(features)

    anomalies = isolation_forest.predict(features)
    data['anomaly_isolation_forest'] = anomalies

    print(data['anomaly_isolation_forest'].value_counts())
else:
    print("Error: Features DataFrame is empty, cannot fit Isolation Forest model.")

Error: Features DataFrame is empty, cannot fit Isolation Forest model.


In [13]:
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report

data = pd.read_csv('CloudWatch_Traffic_Web_Attack.csv')

from sklearn.metrics import confusion_matrix, classification_report

if 'true_labels' in data.columns:
    true_labels = data['true_labels']

    if 'anomaly_isolation_forest' in data.columns:
        print("Isolation Forest Evaluation:")
        print(confusion_matrix(true_labels, data['anomaly_isolation_forest']))
        print(classification_report(true_labels, data['anomaly_isolation_forest']))